# Notebook for generate Qualitative Graphics

### Libraries import

In [1]:
import os
import sys
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt

sys.path.append("../../")

In [2]:
from utils.savers import generate_qq_plot

### Experiment selection

In [3]:
experiment_id = "0006"
root_path = "../../results/Ganomaly_3D/"
for i in sorted(os.listdir(root_path)):
    if experiment_id in i:
        experiment_folder = os.path.join(root_path, i)
experiment_folder

'../../results/Ganomaly_3D/0006_Ganomaly3D-64x64x64x1'

### Errors loading

In [4]:
base_path = os.path.join(experiment_folder, "outputs/errors/")
for t in ["encoder", "contextual", "adversarial"]:
        for c in ["normal", "abnormal"]:
            globals()["all_{}_{}".format(t, c)] = np.r_[[]]

for t in ["encoder", "contextual", "adversarial"]:
    for m in ["train", "val", "test"]:
        if m == "train":
            if os.path.isfile(os.path.join(base_path, t, m, "normal.npy")):
                classes = ["normal"] 
            else:
                classes = ["abnormal"]
        else:
            classes = ["normal", "abnormal"]

        for c in classes:
            all_data = "all_{}_{}".format(t, c)
            errors = np.load(os.path.join(base_path, t, m, c + ".npy"))
            globals()["{}_{}_{}".format(m, t, c)] = errors
            globals()[all_data] = np.concatenate([globals()[all_data], errors])

### Errors by patients loading

In [5]:
base_path = os.path.join(experiment_folder, "outputs/latent_vectors/input_generator")
for t in ["encoder", "contextual", "adversarial"]:
    for m in ["train", "val", "test"]:
        if m == "train":
            if "train_encoder_normal" in globals().keys():
                classes = ["normal"] 
            else:
                classes = ["abnormal"]
        else:
            classes = ["normal", "abnormal"]
        for c in classes:
            patients_ids_positions = [
                int(i.split("_")[1].split("-")[1].split(".")[0]) for i in sorted(
                    os.listdir(os.path.join(base_path, m, c))
                )
            ]
            data = "{}_{}_{}".format(m, t, c)
            key = "{}_{}".format(data, "patients")
            globals()[key] = {}

            for p_id in np.unique(patients_ids_positions):
                globals()[key][p_id] = []

            for i, p_id in enumerate(patients_ids_positions):
                globals()[key][p_id].append(globals()[data][i])

### Data partition distribution individual

In [15]:
save_path = os.path.join(experiment_folder, "outputs/graphics/qualitative/individual_distributions")
if not os.path.isdir(save_path):
    os.mkdir(save_path)
for t in ["encoder", "contextual", "adversarial"]:
    for g in ["train", "val", "test", "all"]:
        if g == "train":
            classes = ["check"]
        else:
            classes = ["normal", "abnormal"]
        for cl in classes:
            if cl == "check":
                if "train_{}_normal".format(t) in globals().keys():
                    c = "normal"
                else:
                    c = "abnormal"
            else:
                c = cl
            data = globals()["{}_{}_{}".format(g, t, c)]
            h = (2*stats.iqr(data)) / (data.shape[0]**(1/3))
            n_bins = int((np.max(data) - np.min(data)) / h)
            # density = stats.gaussian_kde(data)
            plt.figure(figsize=(15,13))
            # n, x, _ = plt.hist(data, bins=n_bins, label="{} {}".format(g, c), histtype="step")
            plt.hist(data, bins=n_bins, label="{} {}".format(g, c), histtype="step")
            # plt.plot(x, density(x))
            plt.xlabel("Error")#, size=14)
            plt.ylabel("Counts")#, size=14)
            plt.title("Data Distribution")
            plt.legend()
            path = os.path.join(save_path, '{}_{}_{}_distribution.png'.format(g, t, c))
            plt.savefig(path)
            plt.close()

### Data partition distribution together

In [13]:
save_path = os.path.join(experiment_folder, "outputs/graphics/qualitative/together_distributions")
if not os.path.isdir(save_path):
    os.mkdir(save_path)
for t in ["encoder", "contextual", "adversarial"]:
    plt.figure(figsize=(15,13))
    for g in ["val", "test"]:
        for c in ["normal", "abnormal"]:
            data = globals()["{}_{}_{}".format(g, t, c)]
            h = (2*stats.iqr(data)) / (data.shape[0]**(1/3))
            n_bins = int((np.max(data) - np.min(data)) / h)
            plt.hist(data, bins=n_bins, label="{} {}".format(g, c), histtype="step")
    plt.xlabel("Errors")#, size=14)
    plt.ylabel("Counts")#, size=14)
    plt.title("Data Distributions")
    plt.legend()
    path = os.path.join(save_path, '{}_distributions.png'.format(t))
    plt.savefig(path)
    plt.close()

### Data partition distribution grouped

In [14]:
save_path = os.path.join(experiment_folder, "outputs/graphics/qualitative/grouped_distributions")
if not os.path.isdir(save_path):
    os.mkdir(save_path)
for t in ["encoder", "contextual", "adversarial"]:
    plt.figure(figsize=(15,13))
    for g in ["val", "test", "all"]:
        for c in ["normal", "abnormal"]:
            data = globals()["{}_{}_{}".format(g, t, c)]
            h = (2*stats.iqr(data)) / (data.shape[0]**(1/3))
            n_bins = int((np.max(data) - np.min(data)) / h)
            plt.hist(data, bins=n_bins, label="{} {}".format(g, c), histtype="step")
        plt.xlabel("Errors")#, size=14)
        plt.ylabel("Counts")#, size=14)
        plt.title("Data Distributions")
        plt.legend()
        path = os.path.join(save_path, '{}_{}_distributions.png'.format(g, t))
        plt.savefig(path)
        plt.close()

### QQ Plots

In [22]:
save_path = os.path.join(experiment_folder, "outputs/graphics/qualitative/qq_plots")
if not os.path.isdir(save_path):
    os.mkdir(save_path)
for t in ["encoder", "contextual", "adversarial"]:
    for m in ["train", "val", "test", "all"]:
        if m == "train":
            if "train_encoder_normal" in globals().keys():
                classes = ["normal"] 
            else:
                classes = ["abnormal"]
        else:
            classes = ["normal", "abnormal"]
        for c in classes:
            data = globals()["{}_{}_{}".format(m, t, c)]
            generate_qq_plot(
                data, 
                save_path, 
                "{}_{}_{}_qqplot".format(m, t, c),
                ".png",
                np.mean(data),
                np.std(data)
            )

### Box Plots

In [18]:
save_path = os.path.join(experiment_folder, "outputs/graphics/qualitative/box_plots")
if not os.path.isdir(save_path):
    os.mkdir(save_path)
rows = 1
columns = 3
for m in ["val", "test", "all"]:
    fig, axs = plt.subplots(rows, columns, figsize=(15, 10))
    for i, t in enumerate(["encoder", "contextual", "adversarial"]):
        data = "{}_{}_".format(m, t)
        axs[i].boxplot([globals()[data+"normal"], globals()[data+"abnormal"]], labels=['Normal', 'Abnormal'])
        axs[i].set_title("{} errors".format(t))
    filename = '{}_boxplot.png'.format(m)
    fig.savefig(os.path.join(save_path, filename))
    plt.close(fig)